In [ ]:
import velodyne_decoder as vd
import pandas as pd
import numpy as np

from Transformation import *
from datetime import datetime
# no scientific notation in jupyter nb
np.set_printoptions(suppress=True,   precision=10,  threshold=2000,  linewidth=150)  
pd.set_option('display.float_format',lambda x : '%.6f' % x)


In [ ]:
# def utc_to_weekseconds(utc_timestamp,leapseconds):
#     """ Returns the GPS week, the GPS day, and the seconds
#         and microseconds since the beginning of the GPS week
#         https://stackoverflow.com/questions/45422739/gps-time-in-weeks-since-epoch-in-python
#     """
#     import datetime, calendar
#     utc = datetime.datetime.fromtimestamp(utc_timestamp)
#     datetimeformat = "%Y-%m-%d %H:%M:%S"
#     epoch = datetime.datetime.strptime("1980-01-06 00:00:00",datetimeformat)
#     tdiff = utc -epoch  + datetime.timedelta(seconds=leapseconds)
#     gpsweek = tdiff.days // 7
#     gpsdays = tdiff.days - 7*gpsweek
#     gpsseconds = tdiff.seconds + 86400* (tdiff.days -7*gpsweek)
#     return gpsweek, gpsdays, gpsseconds, tdiff.microseconds, gpsseconds+(tdiff.microseconds / 10**6)

In [ ]:
# Read the PCAP file and extract the UNIX timestamp
pcap_file = '../data/2022-07-13-15-49-46_Velodyne-HDL-64-Data (Frame 0 to 100).pcap'
config = vd.Config(model='HDL-64E', rpm=600, min_range=3.5, max_range=85)
cloud_arrays = []
timestamp = []
for stamp, points in vd.read_pcap(pcap_file, config=config):
    cloud_arrays.append(points)
    timestamp.append(stamp)

In [ ]:
# Convert to numpy format
timestamp_TPE = np.array(timestamp)
# Convert the local timezone (GMT+ 8 hour) to Greenwich Mean Time (GMT)
timestamp_GMT = timestamp_TPE - 28800
# test 
timestamp_GMT[0]

In [ ]:
# Unix timestamp to GPS seconds 
ts = timestamp_GMT
# Initialize a list to save the convertion data
# Make sure the timestamp is GMT and set the leapsecond as 18
GPS_lidar = []
for i in ts:
    GPS_lidar.append(utc_to_weekseconds(i, 18))

# Convert to numpy format    
GPS_lidar = np.array(GPS_lidar)
GPS_lidar[0]


In [ ]:
# select the GPS second column only
GPS_second_lidar = GPS_lidar[:, 4]
GPS_second_lidar[0]

In [ ]:
# Save the GPS lidar time as csv
header = ['UNIX','GPSsecond']
output = np.stack((ts, GPS_second_lidar), axis=1)
pd.DataFrame(output, columns=header).to_csv('./2022-07-13-HDL64E-timestamp.csv', index=None)


In [ ]:
# Read the RQH data as Pandas format
nav = pd.read_csv('../data/20220713_Highways_RQH_TC.txt',
                            sep ="\s+|\t+|\s+\t+|\t+\s+", engine='python',
                            skiprows=[i for i in range(24)]+[25],   # Skip matadata(0~23) & unit(25) for IE output
                            skipfooter=5,
                            low_memory=True)

In [ ]:
# Test the very first timestamp 
nav.head(5)